### SahabatAI-v1-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install
!git clone https://huggingface.co/Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

In [ ]:
import os
from pathlib import Path

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "/workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct" ############## <--- Change this based on platform and models

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
######### set LLM_MODEL to the same path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

In [ ]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [ ]:
os.environ["LLM_CLIENT_WAIT_TIME"] = "600" # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.
os.environ["LLM_WORKER_MAX_TIME"] = "600"  # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.

!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --max_new_tokens 6700

In [ ]:
os.environ["LLM_CLIENT_WAIT_TIME"] = "1800" 
os.environ["LLM_WORKER_MAX_TIME"] = "1800"

#!python decompose_to_cnf.py --data_path manual_data_translated --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name //workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --max_new_tokens 6700

In [ ]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

In [ ]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation False --max_new_tokens 6700

In [ ]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation True --max_new_tokens 6700

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct